# This notebook is designed for teaching/testing purposes to help you visualize the tensor shapes that go through each module

In [1]:
# my virtual environments are rarely properly connected to jupyter so this fixes that. 
# you prolly won't need this cell but running it won't hurt anything either
import sys
import os
current_dir = os.getcwd()  # Get the current working directory
venv_dir = os.path.join(current_dir, 'venv') 
python_version = str(sys.version_info.major) + '.' + str(sys.version_info.minor)
site_packages_path = os.path.join(venv_dir, 'lib', 'python' + python_version, 'site-packages')
sys.path.append(site_packages_path) 

In [2]:
# config file
from config import ModelConfig
cfg = ModelConfig()
print(cfg)

# import the tokenizer specified by cfg
from tools import import_from_nested_path
imported_objects = import_from_nested_path(['tokenizers', cfg.tokenizer], 'tokenizer', ['get_tokenizer'])
get_tokenizer = imported_objects.get('get_tokenizer')
tokenizer = get_tokenizer(size = cfg.vocab_len) # assuming 'bpe', size options are 95, 128, 256, 512, 1024, 2048, 4096, 8192

import random
import torch

ModelConfig(dim=64, device='cpu', tokenizer='bpe', vocab_len=8192, num_layers=4, second_resid_norm=False, mlp_hidden_mult=8, mlp_bias=False, mlp_nonlinearity='SiLU', mlp_gated=True, num_q_heads=3, num_kv_heads=1, head_dim=64, theta=10000, max_seq_len=512, fractal_split=2, scale_first_resid=True, norm_type='RMSNorm', norm_affine=True, norm_bias=True, eps=1e-06, max_batch_size=1)


# Norms

In [3]:
from modules.norm import Norm

In [4]:
%%time

### RMSNorm

# Create an instance of RMSNorm
module = Norm(cfg.dim, 'RMSNorm')

# let's take a look
print(sum(p.numel() for p in module.parameters())/1e3, 'K parameters')
print(module)

# Initially, logging is disabled by default
module.enable_logging()

### Optionally disabling printing for sub-functions
#module.disable_function_logging('CosineNorm')
#module.disable_function_logging('LayerNorm')
#module.disable_function_logging('RMSNorm')

x = torch.randn(32,cfg.max_seq_len,cfg.dim)

# Call the forward method - logging will occur
output = module(x)

# Disable logging. 
# This isn't actually necessary since we won't be using this object again but that's how you'd do it
module.disable_logging()

# clearing up ram jic we're training later
del module, x, output

0.128 K parameters
Norm()

==========Entering Norm.forward==========
Inputs:
Tensor 'x' shape: torch.Size([32, 512, 64])

==========Entering Norm.RMSNorm==========
Inputs:
Tensor 'x' shape: torch.Size([32, 512, 64])

Outputs:
Tensor 'output' shape: torch.Size([32, 512, 64])
==========Exiting Norm.RMSNorm==========

Outputs:
Tensor 'output' shape: torch.Size([32, 512, 64])
==========Exiting Norm.forward==========
CPU times: user 56.3 ms, sys: 9.23 ms, total: 65.5 ms
Wall time: 405 ms


In [5]:
%%time

# LayerNorm
module = Norm(cfg.dim, 'LayerNorm')
module.enable_logging()

### Optionally disabling printing for sub-functions
#module.disable_function_logging('CosineNorm')
#module.disable_function_logging('LayerNorm')
#module.disable_function_logging('RMSNorm')

x = torch.randn(32,cfg.max_seq_len,cfg.dim)
output = module(x)
module.disable_logging()
del module, x, output


==========Entering Norm.forward==========
Inputs:
Tensor 'x' shape: torch.Size([32, 512, 64])

==========Entering Norm.LayerNorm==========
Inputs:
Tensor 'x' shape: torch.Size([32, 512, 64])

Outputs:
Tensor 'output' shape: torch.Size([32, 512, 64])
==========Exiting Norm.LayerNorm==========

Outputs:
Tensor 'output' shape: torch.Size([32, 512, 64])
==========Exiting Norm.forward==========
CPU times: user 172 ms, sys: 20.9 ms, total: 193 ms
Wall time: 326 ms


# Attention

In [6]:
from modules.fha import FHA, precompute_freqs_cis

In [7]:
%%time

# first up let's look at training

# Create an instance of fractal-head attention
module = FHA(
        cfg.dim,
        cfg.head_dim,
        cfg.num_q_heads,
        cfg.num_kv_heads,
        cfg.max_batch_size,
        cfg.max_seq_len,
        cfg.fractal_split
)
print(sum(p.numel() for p in module.parameters())/1e3, 'K parameters')
print(module)

# Initially, logging is disabled by default
module.enable_logging()

### Optionally disabling printing for sub-functions
#module.disable_function_logging('apply_rotary_emb')
#module.disable_function_logging('reshape_for_broadcast')
#module.disable_function_logging('match_headcount')
#module.disable_function_logging('attend')
#module.disable_function_logging('calc_output')

# precompute RoPE frequencies, causal mask, and dummy input data
freqs_cis = precompute_freqs_cis(
    cfg.head_dim,
    cfg.max_seq_len,
    cfg.theta
)
mask = torch.full(
    (cfg.max_seq_len, cfg.max_seq_len),
    float("-inf"),
    device=cfg.device
)
mask = torch.triu(mask, diagonal=1)
x = torch.randn(32,cfg.max_seq_len,cfg.dim)

# Call the forward method - logging will occur
output = module(x, freqs_cis, mask, training=True)

# Disable logging. 
# This isn't actually necessary since we won't be using this object again but that's how you'd do it
module.disable_logging()

# clearing up ram jic we're training later
del module, freqs_cis, mask, x, output

32.768 K parameters
FHA(
  (Wq): Linear(in_features=64, out_features=192, bias=False)
  (Wk): Linear(in_features=64, out_features=64, bias=False)
  (Wv): Linear(in_features=64, out_features=64, bias=False)
  (Wo): Linear(in_features=192, out_features=64, bias=False)
)

==========Entering FHA.forward==========
Inputs:
Tensor 'x' shape: torch.Size([32, 512, 64])
Tensor 'freqs_cis' shape: torch.Size([512, 32])
Tensor 'mask' shape: torch.Size([512, 512])
Integer 'cache_len': Value=True

==========Entering FHA.apply_rotary_emb==========
Inputs:
Tensor 'xq' shape: torch.Size([32, 512, 3, 64])
Tensor 'xk' shape: torch.Size([32, 512, 1, 64])
Tensor 'freqs_cis' shape: torch.Size([512, 32])

==========Entering FHA.reshape_for_broadcast==========
Inputs:
Tensor 'freqs_cis' shape: torch.Size([512, 32])
Tensor 'x' shape: torch.Size([32, 512, 3, 32])

Outputs:
Tensor 'output' shape: torch.Size([1, 512, 1, 32])
==========Exiting FHA.reshape_for_broadcast==========

Outputs:
Tensor 'output[0]' shape: 

In [8]:
%%time

# now let's do it for inference

# Create an instance of fractal-head attention
module = FHA(
        cfg.dim,
        cfg.head_dim,
        cfg.num_q_heads,
        cfg.num_kv_heads,
        cfg.max_batch_size,
        cfg.max_seq_len,
        cfg.fractal_split
)
print(sum(p.numel() for p in module.parameters())/1e3, 'K parameters')
print(module)

module.enable_logging()
#module.disable_function_logging('apply_rotary_emb')
#module.disable_function_logging('reshape_for_broadcast')
#module.disable_function_logging('match_headcount')
#module.disable_function_logging('attend')
#module.disable_function_logging('calc_output')

# precompute RoPE frequencies, causal mask, and dummy input data
freqs_cis = precompute_freqs_cis(
    cfg.head_dim,
    cfg.max_seq_len,
    cfg.theta
)
mask = torch.full(
    (cfg.max_seq_len, cfg.max_seq_len),
    float("-inf"),
    device=cfg.device
)
mask = torch.triu(mask, diagonal=1)

# setting up for kv caching
context_chunk_len = cfg.max_seq_len // 4
cache_len = random.randint(1, 3 * context_chunk_len)
seq_len = cache_len + context_chunk_len
# need to extend the mask with zeros for the cached values
mask = mask[:context_chunk_len, :context_chunk_len]
mask = torch.hstack(
            [torch.zeros((context_chunk_len, cache_len)), mask]
        )

# these don't use seq_len because those entries should already be in the kv cache
freqs_cis = freqs_cis[:context_chunk_len]
x = torch.randn(cfg.max_batch_size,context_chunk_len,cfg.dim)

# Call the forward method - logging will occur
output = module(x, freqs_cis, mask, cache_len)

# Disable logging. 
# This isn't actually necessary since we won't be using this object again but that's how you'd do it
module.disable_logging()

# clearing up ram jic we're training later
del module, freqs_cis, mask, cache_len, context_chunk_len, seq_len, x, output

32.768 K parameters
FHA(
  (Wq): Linear(in_features=64, out_features=192, bias=False)
  (Wk): Linear(in_features=64, out_features=64, bias=False)
  (Wv): Linear(in_features=64, out_features=64, bias=False)
  (Wo): Linear(in_features=192, out_features=64, bias=False)
)

==========Entering FHA.forward==========
Inputs:
Tensor 'x' shape: torch.Size([1, 128, 64])
Tensor 'freqs_cis' shape: torch.Size([128, 32])
Tensor 'mask' shape: torch.Size([128, 264])
Integer 'cache_len': Value=136

==========Entering FHA.apply_rotary_emb==========
Inputs:
Tensor 'xq' shape: torch.Size([1, 128, 3, 64])
Tensor 'xk' shape: torch.Size([1, 128, 1, 64])
Tensor 'freqs_cis' shape: torch.Size([128, 32])

==========Entering FHA.reshape_for_broadcast==========
Inputs:
Tensor 'freqs_cis' shape: torch.Size([128, 32])
Tensor 'x' shape: torch.Size([1, 128, 3, 32])

Outputs:
Tensor 'output' shape: torch.Size([1, 128, 1, 32])
==========Exiting FHA.reshape_for_broadcast==========

Outputs:
Tensor 'output[0]' shape: torch

# MLP

In [9]:
from modules.mlp import MLP

In [10]:
%%time

# GeGLU
module = MLP(
    cfg.dim, 
    int(cfg.dim * cfg.mlp_hidden_mult * 2/3), 
    cfg.dim, 
    'GeLU', 
    gated=True, 
    bias=False, 
    dropout_rate = 0.1
)
print(sum(p.numel() for p in module.parameters())/1e3, 'K parameters')
print(module)
module.enable_logging()

x = torch.randn(32,cfg.max_seq_len,cfg.dim)
output = module(x, training=True)
module.disable_logging()
del module, x, output

65.472 K parameters
MLP(
  (Wup): Linear(in_features=64, out_features=341, bias=False)
  (Wgate): Linear(in_features=64, out_features=341, bias=False)
  (Wdown): Linear(in_features=341, out_features=64, bias=False)
  (nonlinearity): GELU(approximate='none')
)

==========Entering MLP.forward==========
Inputs:
Tensor 'x' shape: torch.Size([32, 512, 64])
Integer 'training': Value=True

Outputs:
Tensor 'output' shape: torch.Size([32, 512, 64])
==========Exiting MLP.forward==========
CPU times: user 283 ms, sys: 60.4 ms, total: 343 ms
Wall time: 471 ms


In [11]:
%%time

# not gated, testing every other nonlinearity
module = MLP(
    cfg.dim, 
    int(cfg.dim * cfg.mlp_hidden_mult), 
    cfg.dim, 
    'ReLU', 
    gated=False, 
    bias=False, 
    dropout_rate = 0.1
)
print(sum(p.numel() for p in module.parameters())/1e3, 'K parameters')
print(module)
module.enable_logging()

x = torch.randn(32,cfg.max_seq_len,cfg.dim)
output = module(x, training=True)
module.disable_logging()
del module, x, output

65.536 K parameters
MLP(
  (Wup): Linear(in_features=64, out_features=512, bias=False)
  (Wdown): Linear(in_features=512, out_features=64, bias=False)
  (nonlinearity): ReLU()
)

==========Entering MLP.forward==========
Inputs:
Tensor 'x' shape: torch.Size([32, 512, 64])
Integer 'training': Value=True

Outputs:
Tensor 'output' shape: torch.Size([32, 512, 64])
==========Exiting MLP.forward==========
CPU times: user 297 ms, sys: 98.6 ms, total: 395 ms
Wall time: 194 ms


# ResidualLayer

In [12]:
from modules.layer import Layer

In [13]:
%%time

# TRAINING
module = Layer(cfg)
print(sum(p.numel() for p in module.parameters())/1e3, 'K parameters')
print(module)

module.enable_logging()
#module.disable_function_logging('attn_connect')
#module.disable_function_logging('mlp_connect')
### enabling printing for sub-modules
#module.pre_attn_norm.enable_logging()
#module.attn.enable_logging()
#module.post_attn_norm.enable_logging()
#module.pre_mlp_norm.enable_logging()
#module.mlp.enable_logging()
#module.post_mlp_norm.enable_logging()

# precompute RoPE frequencies, causal mask, and dummy input data
freqs_cis = precompute_freqs_cis(
    cfg.head_dim,
    cfg.max_seq_len,
    cfg.theta
)
mask = torch.full(
    (cfg.max_seq_len, cfg.max_seq_len),
    float("-inf"),
    device=cfg.device
)
mask = torch.triu(mask, diagonal=1)
x = torch.randn(32,cfg.max_seq_len,cfg.dim)

output = module(x, freqs_cis, mask, training=True)
module.disable_logging()
del module, x, output

98.496 K parameters
Layer(
  (pre_attn_norm): Norm()
  (attn): FHA(
    (Wq): Linear(in_features=64, out_features=192, bias=False)
    (Wk): Linear(in_features=64, out_features=64, bias=False)
    (Wv): Linear(in_features=64, out_features=64, bias=False)
    (Wo): Linear(in_features=192, out_features=64, bias=False)
  )
  (pre_mlp_norm): Norm()
  (mlp): MLP(
    (Wup): Linear(in_features=64, out_features=341, bias=False)
    (Wgate): Linear(in_features=64, out_features=341, bias=False)
    (Wdown): Linear(in_features=341, out_features=64, bias=False)
    (nonlinearity): SiLU()
  )
)

==========Entering Layer.forward==========
Inputs:
Tensor 'x' shape: torch.Size([32, 512, 64])
Tensor 'freqs_cis' shape: torch.Size([512, 32])
Tensor 'mask' shape: torch.Size([512, 512])
Integer 'cache_len': Value=True

==========Entering Layer.attn_connect==========
Inputs:
Tensor 'x' shape: torch.Size([32, 512, 64])
Tensor 'freqs_cis' shape: torch.Size([512, 32])
Tensor 'mask' shape: torch.Size([512, 512

In [14]:
%%time

# INFERENCE
module = Layer(cfg)
print(sum(p.numel() for p in module.parameters())/1e3, 'K parameters')
print(module)

module.enable_logging()
#module.disable_function_logging('attn_connect')
#module.disable_function_logging('mlp_connect')
#module.pre_attn_norm.enable_logging()
#module.attn.enable_logging()
#module.post_attn_norm.enable_logging()
#module.pre_mlp_norm.enable_logging()
#module.mlp.enable_logging()
#module.post_mlp_norm.enable_logging()

# precompute RoPE frequencies, causal mask, and dummy input data
freqs_cis = precompute_freqs_cis(
    cfg.head_dim,
    cfg.max_seq_len,
    cfg.theta
)
mask = torch.full(
    (cfg.max_seq_len, cfg.max_seq_len),
    float("-inf"),
    device=cfg.device
)
mask = torch.triu(mask, diagonal=1)
# setting up for kv caching
cache_len = cfg.max_seq_len // 3
context_chunk_len = cfg.max_seq_len // 4
seq_len = cache_len + context_chunk_len
# need to extend the mask with zeros for the cached values
mask = mask[:context_chunk_len, :context_chunk_len]
mask = torch.hstack(
            [torch.zeros((context_chunk_len, cache_len)), mask]
        )
# these don't use seq_len because those entries should already be in the kv cache
freqs_cis = freqs_cis[:context_chunk_len]
x = torch.randn(1,context_chunk_len,cfg.dim)

output = module(x, freqs_cis, mask, cache_len)
module.disable_logging()
del module, freqs_cis, mask, cache_len, context_chunk_len, seq_len, x, output

98.496 K parameters
Layer(
  (pre_attn_norm): Norm()
  (attn): FHA(
    (Wq): Linear(in_features=64, out_features=192, bias=False)
    (Wk): Linear(in_features=64, out_features=64, bias=False)
    (Wv): Linear(in_features=64, out_features=64, bias=False)
    (Wo): Linear(in_features=192, out_features=64, bias=False)
  )
  (pre_mlp_norm): Norm()
  (mlp): MLP(
    (Wup): Linear(in_features=64, out_features=341, bias=False)
    (Wgate): Linear(in_features=64, out_features=341, bias=False)
    (Wdown): Linear(in_features=341, out_features=64, bias=False)
    (nonlinearity): SiLU()
  )
)

==========Entering Layer.forward==========
Inputs:
Tensor 'x' shape: torch.Size([1, 128, 64])
Tensor 'freqs_cis' shape: torch.Size([128, 32])
Tensor 'mask' shape: torch.Size([128, 298])
Integer 'cache_len': Value=170

==========Entering Layer.attn_connect==========
Inputs:
Tensor 'x' shape: torch.Size([1, 128, 64])
Tensor 'freqs_cis' shape: torch.Size([128, 32])
Tensor 'mask' shape: torch.Size([128, 298])


# Full Model

In [15]:
from modules.model import Model

In [16]:
%%time

# TRAINING
module = Model(cfg)
print(sum(p.numel() for p in module.parameters())/1e3, 'K parameters')
print(module)

module.enable_logging()
### enabling printing for sub-modules
module.layers[0].enable_logging()
module.final_norm.enable_logging()

input_token_ids = torch.randint(tokenizer.vocab_len, (32, cfg.max_seq_len))
target_token_ids = torch.randint(tokenizer.vocab_len, (32, cfg.max_seq_len))

output, loss = module(input_token_ids, target_token_ids=target_token_ids)
print(loss)
module.disable_logging()
del module, input_token_ids, target_token_ids, output

918.592 K parameters
Model(
  (token_embedder): Embedding(8195, 64)
  (layers): ModuleList(
    (0-3): 4 x Layer(
      (pre_attn_norm): Norm()
      (attn): FHA(
        (Wq): Linear(in_features=64, out_features=192, bias=False)
        (Wk): Linear(in_features=64, out_features=64, bias=False)
        (Wv): Linear(in_features=64, out_features=64, bias=False)
        (Wo): Linear(in_features=192, out_features=64, bias=False)
      )
      (pre_mlp_norm): Norm()
      (mlp): MLP(
        (Wup): Linear(in_features=64, out_features=341, bias=False)
        (Wgate): Linear(in_features=64, out_features=341, bias=False)
        (Wdown): Linear(in_features=341, out_features=64, bias=False)
        (nonlinearity): SiLU()
      )
    )
  )
  (final_norm): Norm()
  (criterion): CrossEntropyLoss()
)

==========Entering Model.forward==========
Inputs:
Tensor 'input_token_ids' shape: torch.Size([32, 512])
Tensor 'cache_len' shape: torch.Size([32, 512])

==========Entering Layer.forward==========
In

In [17]:
%%time

# Inference
module = Model(cfg)
print(sum(p.numel() for p in module.parameters())/1e3, 'K parameters')
print(module)

module.enable_logging()
### enabling printing for sub-modules
#for i in range(cfg.num_layers):
    #module.layers[i].enable_logging()
#module.final_norm.enable_logging()

input_token_ids = torch.randint(tokenizer.vocab_len, (1, cfg.max_seq_len // 4))

output, _ = module(input_token_ids, cache_len = cfg.max_seq_len // 3)
print(output.shape)
module.disable_logging()
del module, input_token_ids, output

918.592 K parameters
Model(
  (token_embedder): Embedding(8195, 64)
  (layers): ModuleList(
    (0-3): 4 x Layer(
      (pre_attn_norm): Norm()
      (attn): FHA(
        (Wq): Linear(in_features=64, out_features=192, bias=False)
        (Wk): Linear(in_features=64, out_features=64, bias=False)
        (Wv): Linear(in_features=64, out_features=64, bias=False)
        (Wo): Linear(in_features=192, out_features=64, bias=False)
      )
      (pre_mlp_norm): Norm()
      (mlp): MLP(
        (Wup): Linear(in_features=64, out_features=341, bias=False)
        (Wgate): Linear(in_features=64, out_features=341, bias=False)
        (Wdown): Linear(in_features=341, out_features=64, bias=False)
        (nonlinearity): SiLU()
      )
    )
  )
  (final_norm): Norm()
  (criterion): CrossEntropyLoss()
)

==========Entering Model.forward==========
Inputs:
Tensor 'input_token_ids' shape: torch.Size([1, 128])
Integer 'cache_len': Value=170

Outputs:
Tensor 'output[0]' shape: torch.Size([1, 128, 8195])
O